### Criando a classe dos dados:

In [1]:
import random
import json

In [2]:
class Sentiment:
  NEGATIVE = "NEGATIVE"
  NEUTRAL = "NEUTRAL"
  POSITIVE = "POSITIVE"

In [3]:
class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score
    self.sentiment = self.get_sentiment()

  def get_sentiment(self):
    if self.score <= 2:
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else: #score = 4 ou 5
      return Sentiment.POSITIVE

### Carregando os dados:

In [4]:
file_name = 'data/Books_small.json'

reviews = []

with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].text)
print(reviews[5].score)

Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!
4.0


### Preparando os dados

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

training[0].sentiment

'POSITIVE'

In [7]:
train_x = [x.text for x in training]
train_y = [y.sentiment for y in training]

test_x = [x.text for x in test]
test_y = [y.sentiment for y in test]

print(train_x[0])
print(train_y[0])

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
POSITIVE


##### Vetorização utilizando Bags of Words

&emsp;&emsp;&emsp;&emsp;This is an example.

![alt text](Screenshot_1.jpg "Title")

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# criando uma matriz que represente os textos da forma descrita acima
train_x_vectors = vectorizer.fit_transform(train_x)

print(train_x[0])
print(train_x_vectors[0])

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 17 stored elements and shape (1, 7372)>
  Coords	Values
  (0, 7086)	1
  (0, 1148)	1
  (0, 350)	2
  (0, 1800)	1
  (0, 6595)	1
  (0, 562)	1
  (0, 3054)	1
  (0, 1558)	1
  (0, 6475)	1
  (0, 6593)	1
  (0, 2895)	1
  (0, 7353)	1
  (0, 539)	1
  (0, 1515)	1
  (0, 5197)	1
  (0, 3545)	1
  (0, 2007)	1
